# LIME como método XAI

## Inteligencia Artificial
## Grado en Ingeniería Informática - Ingeniería del Software
## Universidad de Sevilla

### Entrenamiento de redes neuronales y árboles de decisión RandomForest para diferentes conjuntos de datos

Lo primero que vamos a hacer es importar diferentes librerías necesarias para crear una red neuronal. Todo lo obtenido es a partir de la práctica sobre redes neuronales realizada en clase de laboratorio.

In [1]:
#Importamos las librerias necesarias para crear la red neuronal
import pandas
import numpy as np
from sklearn import model_selection, preprocessing
from tensorflow import keras
import tensorflow as tf
#Leemos el csv
evaluation = pandas.read_csv('turkiye-student-evaluation_R_Specific.csv', header=0)

In [2]:
#Mostramos las 10 primeras lineas para ver que se ha leido bien
evaluation.head(10)

,instr,class,nb.repeat,attendance,difficulty,Q1,Q2,Q3,Q4,Q5,...,Q19,Q20,Q21,Q22,Q23,Q24,Q25,Q26,Q27,Q28
1,1,2,1,0,4,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
2,1,2,1,1,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
3,1,2,1,2,4,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
4,1,2,1,1,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
5,1,2,1,0,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
6,1,2,1,3,3,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
7,1,2,1,1,3,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
8,1,2,1,1,3,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
9,1,2,1,1,3,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
10,1,2,1,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4


Escogemos dentro de los datos las columnas que vamos a utilizar como atributos

In [3]:
#Seleccionamos los atributos 
atributos = evaluation.loc[:, 'class':'Q27']
atributos = atributos.to_numpy()
scaler = preprocessing.MinMaxScaler()
atributos = scaler.fit_transform(atributos)
print(atributos[5818,:])

[1.   0.   0.25 0.25 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.  ]


Escogemos dentro de los datos la columna, que es la última, que vamos a utilizar como objetivo

In [4]:
#Seleccionamos el objetivo
objetivo = evaluation['Q28']
objetivo = evaluation['Q28'].to_numpy(dtype=np.float32)
print(objetivo)

[3. 3. 5. ... 5. 1. 1.]


Dividimos el conjunto de datos en un subconjunto de entrenamiento y otro de prueba

In [5]:
#Separamos los datos que se usaran para entrenar y los que se usarán para evaluar los modelos
(atributos_entrenamiento, atributos_prueba,
 objetivo_entrenamiento, objetivo_prueba) = model_selection.train_test_split(
    atributos, objetivo, test_size=.15)

Definimos la red neuronal con 31 neuronas de entrada, una por cada atributo, aplicamos un normalizador y definimos 70 neuronas intermedias aplicando la función de activación relu, otras 70 neuronas intermedias aplicando la función de activación sigmoide y como neurona de salida aplicando la función de activación lineal

In [6]:
normalizador = keras.layers.Normalization()#Definimos un normalizador
normalizador.adapt(atributos_entrenamiento)
red_evaluation = keras.Sequential()#Creamos la red neuronal
red_evaluation.add(keras.layers.Input(shape=(31,)))#Definimos la capa de entrada con una neurona por cada atributo
red_evaluation.add(normalizador)#Aplicamos el normalizador
red_evaluation.add(keras.layers.Dense(70, activation='relu'))#Definimos una capa con 70 neuronas y función de activación relu
red_evaluation.add(keras.layers.Dense(70, activation='sigmoid'))#Definimos una capa de 70 neuronas y función de activación sigmoide
red_evaluation.add(keras.layers.Dense(1, activation='linear')) #Defnimios una capa de salida con función de activación lienal ya que los valores esperados son números lineales entre 0 y 5

In [7]:
red_evaluation.summary()#Mostramos un resumen de la red neuronal

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizatio  (None, 31)               63        
 n)                                                              
                                                                 
 dense (Dense)               (None, 70)                2240      
                                                                 
 dense_1 (Dense)             (None, 70)                4970      
                                                                 
 dense_2 (Dense)             (None, 1)                 71        
                                                                 
Total params: 7,344
Trainable params: 7,281
Non-trainable params: 63
_________________________________________________________________


Antes de entrenar la red neuronal, la compilamos aplicando un optimizador (descenso estocástico por el gradiente, SGD), el error cuadrático medio como función de pérdida y medimos las métricas de precisión y el error absoluto medio

In [8]:
red_evaluation.compile(optimizer='SGD', loss='mean_squared_error', metrics=['accuracy', 'mae'])#Compilamos la red neuronal definiendo el optimizaodr, la función de perdida y dos metricas
red_evaluation.fit(atributos_entrenamiento, objetivo_entrenamiento,
                batch_size=256, epochs=10)#Entrenamos la red neuronal con 10 epocas y lotes de 256 elementos

Epoch 1/10
20/20 [==============================] - 1s 2ms/step - loss: 2.3030 - accuracy: 0.1350 - mae: 1.1152  
Epoch 2/10
20/20 [==============================] - 0s 2ms/step - loss: 0.4044 - accuracy: 0.1407 - mae: 0.4619
Epoch 3/10
20/20 [==============================] - 0s 2ms/step - loss: 0.3080 - accuracy: 0.1407 - mae: 0.3582
Epoch 4/10
20/20 [==============================] - 0s 2ms/step - loss: 0.2925 - accuracy: 0.1407 - mae: 0.3335
Epoch 5/10
20/20 [==============================] - 0s 2ms/step - loss: 0.2853 - accuracy: 0.1407 - mae: 0.3213
Epoch 6/10
20/20 [==============================] - 0s 2ms/step - loss: 0.2797 - accuracy: 0.1407 - mae: 0.3159
Epoch 7/10
20/20 [==============================] - 0s 2ms/step - loss: 0.2745 - accuracy: 0.1407 - mae: 0.3080
Epoch 8/10
20/20 [==============================] - 0s 2ms/step - loss: 0.2701 - accuracy: 0.1407 - mae: 0.3044
Epoch 9/10
20/20 [==============================] - 0s 2ms/step - loss: 0.2659 - accuracy: 0.1407 - ma

Buscamos cuál ha sido el entrenamiento con menor pérdida

In [9]:
#Definimos cuanta perdida queremos tener y ejecutamos el siguiente bucle para entrenar la red hasta conseguir una perdida menor a la definida
#Para este conjunto de datos esto practicamente no es necesario pero así nos aseguramos de cometer un error muy pequeño
perdObj = 1.0
x=red_evaluation.evaluate(atributos_prueba, objetivo_prueba)
while x[0]>perdObj:
    red_evaluation.fit(atributos_entrenamiento, objetivo_entrenamiento,
                batch_size=256, epochs=10)
    x=red_evaluation.evaluate(atributos_prueba, objetivo_prueba)
    
print(x[0])

28/28 [==============================] - 0s 1ms/step - loss: 0.2377 - accuracy: 0.1329 - mae: 0.2843
0.23773322999477386


In [10]:
#Ahora que ya hemos conseguido una perdida menor a la que queriamos, evaluamos para volver a comprobar que la hemos conseguido
red_evaluation.evaluate(atributos_prueba, objetivo_prueba)

28/28 [==============================] - 0s 2ms/step - loss: 0.2377 - accuracy: 0.1329 - mae: 0.2843


[0.23773322999477386, 0.13287514448165894, 0.2842792570590973]

Realizamos una prueba manual de las predicciones obtenidas para ver si se asemejan al dato inicial

In [11]:
#Ahora mismo comprobaremos manualmente si las predicciones de la red se parecen a lo que esperabamos utilizando los ultimos registros con los cuales no se ha entrenado
X=atributos[5809]
predicciones = red_evaluation.predict(X)
print('Esperado 4:',predicciones)
X=atributos[5810]
predicciones = red_evaluation.predict(X)
print('Esperado 3:',predicciones)
X=atributos[5811]
predicciones = red_evaluation.predict(X)
print('Esperado 3:',predicciones)
X=atributos[5812]
predicciones = red_evaluation.predict(X)
print('Esperado 1:',predicciones)
X=atributos[5813]
predicciones = red_evaluation.predict(X)
print('Esperado 3:',predicciones)
X=atributos[5814]
predicciones = red_evaluation.predict(X)
print('Esperado 1:',predicciones)
X=atributos[5815]
predicciones = red_evaluation.predict(X)
print('Esperado 1:',predicciones)
X=atributos[5816]
predicciones = red_evaluation.predict(X)
print('Esperado 5:',predicciones)
X=atributos[5817]
predicciones = red_evaluation.predict(X)
print('Esperado 5:',predicciones)
X=atributos[5818]
predicciones = red_evaluation.predict(X)
print('Esperado 1:',predicciones)
X=atributos[5819]
predicciones = red_evaluation.predict(X)
print('Esperado 1:',predicciones)

1/1 [==============================] - 0s 119ms/step
Esperado 4: [[4.1294403]]
1/1 [==============================] - 0s 27ms/step
Esperado 3: [[3.1646411]]
1/1 [==============================] - 0s 30ms/step
Esperado 3: [[3.3052573]]
1/1 [==============================] - 0s 32ms/step
Esperado 1: [[1.1000209]]
1/1 [==============================] - 0s 25ms/step
Esperado 3: [[3.147836]]
1/1 [==============================] - 0s 22ms/step
Esperado 1: [[1.3044006]]
1/1 [==============================] - 0s 28ms/step
Esperado 1: [[1.1000209]]
1/1 [==============================] - 0s 21ms/step
Esperado 5: [[4.7797003]]
1/1 [==============================] - 0s 22ms/step
Esperado 5: [[4.9137855]]
1/1 [==============================] - 0s 23ms/step
Esperado 1: [[1.1374854]]
1/1 [==============================] - 0s 35ms/step
Esperado 1: [[1.1374854]]


Vamos a realizar el estudio con un modelo de árboles de decisión (RandomForest). Por ello, primero vamos a realizar la predicción a partir de un modelo de tipo RandomForestRegressor() y calculamos su pérdida

In [12]:
#Ahora trabajaremos con el otro modelo que será un random forest
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
forest_evaluation = RandomForestRegressor() #Creamos el modelo
forest_evaluation.fit(atributos_entrenamiento, objetivo_entrenamiento)#Lo entrenamos
evaluaciones = forest_evaluation.predict(atributos_prueba)#Evaluamos
mse = mean_squared_error(objetivo_prueba, evaluaciones)#Calculamos la perdida
print("Error cuadrático medio (MSE) del modelo:", mse)
#Como este modelo suele tener una perdida muy pequeña tras entrenarlo una vez, no hacemos nada más para mejorarlo

Error cuadrático medio (MSE) del modelo: 0.19123640201071726


Comprobamos de manera manual si las predicciones obtenidas se asemejan a lo esperado

In [13]:
#Ahora mismo comprobaremos manualmente si las predicciones del random forest se parecen a lo que esperabamos utilizando los ultimos registros con los cuales no se ha entrenado
atributos2 = atributos[:, np.newaxis]
X=atributos2[5809]
predicciones = forest_evaluation.predict(X)
print('Esperado 4:',predicciones)
X=atributos2[5810]
predicciones = forest_evaluation.predict(X)
print('Esperado 3:',predicciones)
X=atributos2[5811]
predicciones = forest_evaluation.predict(X)
print('Esperado 3:',predicciones)
X=atributos2[5812]
predicciones = forest_evaluation.predict(X)
print('Esperado 1:',predicciones)
X=atributos2[5813]
predicciones = forest_evaluation.predict(X)
print('Esperado 3:',predicciones)
X=atributos2[5814]
predicciones = forest_evaluation.predict(X)
print('Esperado 1:',predicciones)
X=atributos2[5815]
predicciones = forest_evaluation.predict(X)
print('Esperado 1:',predicciones)
X=atributos2[5816]
predicciones = forest_evaluation.predict(X)
print('Esperado 5:',predicciones)
X=atributos2[5817]
predicciones = forest_evaluation.predict(X)
print('Esperado 5:',predicciones)
X=atributos2[5818]
predicciones = forest_evaluation.predict(X)
print('Esperado 1:',predicciones)
X=atributos2[5819]
predicciones = forest_evaluation.predict(X)
print('Esperado 1:',predicciones)

Esperado 4: [4.]
Esperado 3: [3.]
Esperado 3: [3.]
Esperado 1: [1.]
Esperado 3: [3.]
Esperado 1: [1.01]
Esperado 1: [1.]
Esperado 5: [4.95]
Esperado 5: [5.]
Esperado 1: [1.]
Esperado 1: [1.]


Realizamos todos los mismos pasos explicados anteriormente pero para otro conjunto de datos

In [14]:
#Importamos las librerias necesarias para crear otra red neuronal
#Volemos a importar por si se quiere ejecutar este fragmento sin ejecutar el anterior
import pandas
import numpy as np
from sklearn import model_selection, preprocessing
from tensorflow import keras
import tensorflow as tf

#Leemos el csv
#bikes = pandas.read_csv('hour.csv', header=0)
bikes = pandas.read_csv('hour2.csv',header = 0)


In [15]:
#Mostramos las 10 primeras lineas para ver que se ha leido bién
bikes.head(10)

,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0000,3,13,16
1,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0000,8,32,40
2,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0000,5,27,32
3,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0000,3,10,13
4,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0000,0,1,1
5,1,0,1,5,0,6,0,2,0.24,0.2576,0.75,0.0896,0,1,1
6,1,0,1,6,0,6,0,1,0.22,0.2727,0.80,0.0000,2,0,2
7,1,0,1,7,0,6,0,1,0.20,0.2576,0.86,0.0000,1,2,3
8,1,0,1,8,0,6,0,1,0.24,0.2879,0.75,0.0000,1,7,8
9,1,0,1,9,0,6,0,1,0.32,0.3485,0.76,0.0000,8,6,14


In [16]:
#Seleccionamos los atributos y los nomalizamos
attr = bikes.loc[:, 'season':'registered']
attr = attr.to_numpy()
scaler = preprocessing.MinMaxScaler()
attr = scaler.fit_transform(attr)

print(attr)

[[0.         0.         0.         ... 0.         0.00817439 0.01467269]
 [0.         0.         0.         ... 0.         0.02179837 0.03611738]
 [0.         0.         0.         ... 0.         0.01362398 0.03047404]
 ...
 [0.         1.         1.         ... 0.19301751 0.01907357 0.09367946]
 [0.         1.         1.         ... 0.15786999 0.03542234 0.05417607]
 [0.         1.         1.         ... 0.15786999 0.03269755 0.04176072]]


In [17]:
#Seleccionamos los objetivos
obj = bikes['cnt']
obj = bikes['cnt'].to_numpy(dtype=np.float32)
print(obj)

[16. 40. 32. ... 90. 61. 49.]


In [18]:
#Cogemos un 85% de los datos para entrenar y un 15% de los datos para evaluar el modelo
(attr_entrenamiento, attr_prueba,
 obj_entrenamiento, obj_prueba) = model_selection.train_test_split(
    attr, obj, test_size=.10)

In [19]:
normalizador = keras.layers.Normalization()#Definimos un normalizador
normalizador.adapt(attr_entrenamiento)
red_bikes = keras.Sequential()#Creamos la red neuronal
red_bikes.add(keras.layers.Input(shape=(14,))) #Añadimos la capa de entrada con 14 neuronas, una por cada atributo
red_bikes.add(normalizador)#Aplicamos un normalizador
red_bikes.add(keras.layers.Dense(100, activation='sigmoid'))#Añadimos una capa intermedia de 100 neuronas con función de activación sigmoide
red_bikes.add(keras.layers.Dense(100, activation='sigmoid'))#Añadimos una capa intermedia de 100 neuronas con función de activación sigmoide
red_bikes.add(keras.layers.Dense(1, activation='linear'))#Necesitamos una capa de salida linear ya que los valores objetivos son lineales

In [20]:
#Vemos un resumen de la red
red_bikes.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_1 (Normalizat  (None, 14)               29        
 ion)                                                            
                                                                 
 dense_3 (Dense)             (None, 100)               1500      
                                                                 
 dense_4 (Dense)             (None, 100)               10100     
                                                                 
 dense_5 (Dense)             (None, 1)                 101       
                                                                 
Total params: 11,730
Trainable params: 11,701
Non-trainable params: 29
_________________________________________________________________


In [21]:
#Compilamos y entrenamos la red neuronal, buscamos minimizar la función de perdida
red_bikes.compile(optimizer='SGD', loss='mean_squared_error', metrics=['accuracy', 'mae'])
red_bikes.fit(attr_entrenamiento, obj_entrenamiento,
                batch_size=256, epochs=10)

Epoch 1/10
62/62 [==============================] - 0s 1ms/step - loss: 13759.2949 - accuracy: 0.0090 - mae: 84.0571 
Epoch 2/10
62/62 [==============================] - 0s 1ms/step - loss: 2808.4453 - accuracy: 0.0088 - mae: 38.9654
Epoch 3/10
62/62 [==============================] - 0s 1ms/step - loss: 1617.7365 - accuracy: 0.0091 - mae: 31.5200
Epoch 4/10
62/62 [==============================] - 0s 1ms/step - loss: 910.7053 - accuracy: 0.0091 - mae: 23.2298
Epoch 5/10
62/62 [==============================] - 0s 1ms/step - loss: 928.1590 - accuracy: 0.0090 - mae: 23.4361
Epoch 6/10
62/62 [==============================] - 0s 1ms/step - loss: 461.6394 - accuracy: 0.0091 - mae: 16.2470
Epoch 7/10
62/62 [==============================] - 0s 1ms/step - loss: 392.4432 - accuracy: 0.0088 - mae: 13.9908
Epoch 8/10
62/62 [==============================] - 0s 1ms/step - loss: 403.4373 - accuracy: 0.0091 - mae: 15.9092
Epoch 9/10
62/62 [==============================] - 0s 1ms/step - loss: 235

In [22]:
#Definimos cuanta perdida queremos tener y ejecutamos el siguiente bucle para entrenar la red hasta conseguir una perdida menor a la definida
perdidaObj = 1.0
x=red_bikes.evaluate(attr_prueba, obj_prueba)
while x[0]>perdidaObj:
    red_bikes.fit(attr_entrenamiento, obj_entrenamiento,
                batch_size=256, epochs=10)
    x=red_bikes.evaluate(attr_prueba, obj_prueba)
    
#La forma más efectiva que hemos encontrado para mejorar nuestra red ha sido la de hacer este bulce ya que es la unica forma de asegurarnos que la red obtendrá la precisión que busquemos
print(x[0])

55/55 [==============================] - 0s 1ms/step - loss: 85.6212 - accuracy: 0.0086 - mae: 6.4170
Epoch 1/10
62/62 [==============================] - 0s 1ms/step - loss: 228.7481 - accuracy: 0.0091 - mae: 11.2170
Epoch 2/10
62/62 [==============================] - 0s 1ms/step - loss: 161.2271 - accuracy: 0.0090 - mae: 9.6498
Epoch 3/10
62/62 [==============================] - 0s 1ms/step - loss: 154.3520 - accuracy: 0.0086 - mae: 8.2316
Epoch 4/10
62/62 [==============================] - 0s 1ms/step - loss: 272.0161 - accuracy: 0.0091 - mae: 11.1519
Epoch 5/10
62/62 [==============================] - 0s 1ms/step - loss: 140.2182 - accuracy: 0.0091 - mae: 8.2057
Epoch 6/10
62/62 [==============================] - 0s 2ms/step - loss: 303.2591 - accuracy: 0.0087 - mae: 12.9699
Epoch 7/10
62/62 [==============================] - 0s 2ms/step - loss: 170.9122 - accuracy: 0.0091 - mae: 9.7196
Epoch 8/10
62/62 [==============================] - 0s 2ms/step - loss: 195.5923 - accuracy: 0.00

62/62 [==============================] - 0s 2ms/step - loss: 10.9710 - accuracy: 0.0091 - mae: 2.6611
Epoch 3/10
62/62 [==============================] - 0s 2ms/step - loss: 37.0037 - accuracy: 0.0091 - mae: 5.1157
Epoch 4/10
62/62 [==============================] - 0s 2ms/step - loss: 36.8886 - accuracy: 0.0088 - mae: 5.0158
Epoch 5/10
62/62 [==============================] - 0s 2ms/step - loss: 14.4252 - accuracy: 0.0091 - mae: 3.0951
Epoch 6/10
62/62 [==============================] - 0s 2ms/step - loss: 37.9802 - accuracy: 0.0091 - mae: 5.3350
Epoch 7/10
62/62 [==============================] - 0s 2ms/step - loss: 7.4366 - accuracy: 0.0091 - mae: 2.1653
Epoch 8/10
62/62 [==============================] - 0s 2ms/step - loss: 20.2781 - accuracy: 0.0091 - mae: 3.6665
Epoch 9/10
62/62 [==============================] - 0s 2ms/step - loss: 53.8237 - accuracy: 0.0091 - mae: 6.2411
Epoch 10/10
55/55 [==============================] - 0s 1ms/step - loss: 28.1512 - accuracy: 0.0086 - mae: 4

62/62 [==============================] - 0s 2ms/step - loss: 19.2736 - accuracy: 0.0091 - mae: 3.6816
Epoch 6/10
62/62 [==============================] - 0s 2ms/step - loss: 25.1184 - accuracy: 0.0091 - mae: 4.2636
Epoch 7/10
62/62 [==============================] - 0s 2ms/step - loss: 14.5997 - accuracy: 0.0091 - mae: 3.2349
Epoch 8/10
62/62 [==============================] - 0s 2ms/step - loss: 13.7116 - accuracy: 0.0091 - mae: 3.0717
Epoch 9/10
62/62 [==============================] - 0s 2ms/step - loss: 12.4307 - accuracy: 0.0091 - mae: 2.9353
Epoch 10/10
55/55 [==============================] - 0s 872us/step - loss: 10.3367 - accuracy: 0.0086 - mae: 2.3495
Epoch 1/10
62/62 [==============================] - 0s 2ms/step - loss: 35.1895 - accuracy: 0.0091 - mae: 5.0906
Epoch 2/10
62/62 [==============================] - 0s 2ms/step - loss: 7.8071 - accuracy: 0.0091 - mae: 2.1542
Epoch 3/10
62/62 [==============================] - 0s 2ms/step - loss: 12.8554 - accuracy: 0.0091 - mae:

62/62 [==============================] - 0s 2ms/step - loss: 11.8370 - accuracy: 0.0091 - mae: 2.9395
Epoch 9/10
62/62 [==============================] - 0s 2ms/step - loss: 8.2204 - accuracy: 0.0091 - mae: 2.3811
Epoch 10/10
55/55 [==============================] - 0s 1ms/step - loss: 22.3950 - accuracy: 0.0086 - mae: 3.8260
Epoch 1/10
62/62 [==============================] - 0s 2ms/step - loss: 21.2158 - accuracy: 0.0091 - mae: 3.9962
Epoch 2/10
62/62 [==============================] - 0s 2ms/step - loss: 11.7660 - accuracy: 0.0091 - mae: 2.9677
Epoch 3/10
62/62 [==============================] - 0s 2ms/step - loss: 21.4831 - accuracy: 0.0091 - mae: 4.0923
Epoch 4/10
62/62 [==============================] - 0s 2ms/step - loss: 17.1818 - accuracy: 0.0091 - mae: 3.5272
Epoch 5/10
62/62 [==============================] - 0s 2ms/step - loss: 4.6018 - accuracy: 0.0091 - mae: 1.7157
Epoch 6/10
62/62 [==============================] - 0s 2ms/step - loss: 8.7943 - accuracy: 0.0091 - mae: 2.5

55/55 [==============================] - 0s 1ms/step - loss: 18.7633 - accuracy: 0.0086 - mae: 3.5866
Epoch 1/10
62/62 [==============================] - 0s 2ms/step - loss: 9.6277 - accuracy: 0.0091 - mae: 2.6953
Epoch 2/10
62/62 [==============================] - 0s 2ms/step - loss: 12.2650 - accuracy: 0.0091 - mae: 3.0713
Epoch 3/10
62/62 [==============================] - 0s 2ms/step - loss: 15.4367 - accuracy: 0.0091 - mae: 3.4619
Epoch 4/10
62/62 [==============================] - 0s 2ms/step - loss: 4.6285 - accuracy: 0.0091 - mae: 1.7711
Epoch 5/10
62/62 [==============================] - 0s 2ms/step - loss: 11.6334 - accuracy: 0.0091 - mae: 2.9868
Epoch 6/10
62/62 [==============================] - 0s 2ms/step - loss: 17.2766 - accuracy: 0.0091 - mae: 3.6246
Epoch 7/10
62/62 [==============================] - 0s 2ms/step - loss: 14.6717 - accuracy: 0.0091 - mae: 3.2627
Epoch 8/10
62/62 [==============================] - 0s 2ms/step - loss: 4.6305 - accuracy: 0.0091 - mae: 1.74

...

62/62 [==============================] - 0s 2ms/step - loss: 3.3451 - accuracy: 0.0091 - mae: 1.4904
Epoch 7/10
62/62 [==============================] - 0s 2ms/step - loss: 12.5041 - accuracy: 0.0090 - mae: 2.9983
Epoch 8/10
62/62 [==============================] - 0s 2ms/step - loss: 15.4894 - accuracy: 0.0090 - mae: 3.3962
Epoch 9/10
62/62 [==============================] - 0s 2ms/step - loss: 14.7632 - accuracy: 0.0091 - mae: 3.4855
Epoch 10/10
55/55 [==============================] - 0s 1ms/step - loss: 2.6754 - accuracy: 0.0086 - mae: 1.2708
Epoch 1/10
62/62 [==============================] - 0s 2ms/step - loss: 1.8912 - accuracy: 0.0091 - mae: 1.1060
Epoch 2/10
62/62 [==============================] - 0s 2ms/step - loss: 1.3337 - accuracy: 0.0091 - mae: 0.9408
Epoch 3/10
62/62 [==============================] - 0s 2ms/step - loss: 5.5842 - accuracy: 0.0091 - mae: 2.0625
Epoch 4/10
62/62 [==============================] - 0s 2ms/step - loss: 16.1633 - accuracy: 0.0091 - mae: 3.674

62/62 [==============================] - 0s 2ms/step - loss: 3.0418 - accuracy: 0.0091 - mae: 1.4494
Epoch 10/10
55/55 [==============================] - 0s 1ms/step - loss: 3.1762 - accuracy: 0.0086 - mae: 1.3210
Epoch 1/10
62/62 [==============================] - 0s 2ms/step - loss: 4.7476 - accuracy: 0.0091 - mae: 1.8614
Epoch 2/10
62/62 [==============================] - 0s 2ms/step - loss: 5.1722 - accuracy: 0.0091 - mae: 1.9454
Epoch 3/10
62/62 [==============================] - 0s 2ms/step - loss: 7.6704 - accuracy: 0.0091 - mae: 2.4619
Epoch 4/10
62/62 [==============================] - 0s 2ms/step - loss: 9.1965 - accuracy: 0.0091 - mae: 2.7281
Epoch 5/10
62/62 [==============================] - 0s 2ms/step - loss: 11.2946 - accuracy: 0.0091 - mae: 3.0075
Epoch 6/10
62/62 [==============================] - 0s 2ms/step - loss: 13.2872 - accuracy: 0.0091 - mae: 3.3017
Epoch 7/10
62/62 [==============================] - 0s 2ms/step - loss: 4.9412 - accuracy: 0.0091 - mae: 1.8280


62/62 [==============================] - 0s 2ms/step - loss: 5.7712 - accuracy: 0.0091 - mae: 2.0312
Epoch 2/10
62/62 [==============================] - 0s 2ms/step - loss: 5.0815 - accuracy: 0.0091 - mae: 1.9319
Epoch 3/10
62/62 [==============================] - 0s 2ms/step - loss: 1.1422 - accuracy: 0.0091 - mae: 0.8619
Epoch 4/10
62/62 [==============================] - 0s 2ms/step - loss: 7.1474 - accuracy: 0.0091 - mae: 2.3458
Epoch 5/10
62/62 [==============================] - 0s 2ms/step - loss: 2.6800 - accuracy: 0.0091 - mae: 1.3485
Epoch 6/10
62/62 [==============================] - 0s 2ms/step - loss: 5.2250 - accuracy: 0.0091 - mae: 1.9947
Epoch 7/10
62/62 [==============================] - 0s 2ms/step - loss: 12.6862 - accuracy: 0.0091 - mae: 3.2496
Epoch 8/10
62/62 [==============================] - 0s 2ms/step - loss: 10.3903 - accuracy: 0.0091 - mae: 2.8875
Epoch 9/10
62/62 [==============================] - 0s 2ms/step - loss: 4.9499 - accuracy: 0.0091 - mae: 1.8814
E

62/62 [==============================] - 0s 2ms/step - loss: 0.8809 - accuracy: 0.0091 - mae: 0.7757
Epoch 5/10
62/62 [==============================] - 0s 2ms/step - loss: 1.1780 - accuracy: 0.0091 - mae: 0.8795
Epoch 6/10
62/62 [==============================] - 0s 3ms/step - loss: 3.1773 - accuracy: 0.0091 - mae: 1.4719
Epoch 7/10
62/62 [==============================] - 0s 3ms/step - loss: 1.0507 - accuracy: 0.0091 - mae: 0.8251
Epoch 8/10
62/62 [==============================] - 0s 2ms/step - loss: 3.9796 - accuracy: 0.0091 - mae: 1.6698
Epoch 9/10
62/62 [==============================] - 0s 2ms/step - loss: 9.1835 - accuracy: 0.0091 - mae: 2.7137
Epoch 10/10
55/55 [==============================] - 0s 2ms/step - loss: 12.9830 - accuracy: 0.0086 - mae: 2.6470
Epoch 1/10
62/62 [==============================] - 0s 2ms/step - loss: 6.6661 - accuracy: 0.0091 - mae: 2.2778
Epoch 2/10
62/62 [==============================] - 0s 2ms/step - loss: 5.3716 - accuracy: 0.0091 - mae: 2.0604
E

62/62 [==============================] - 0s 2ms/step - loss: 4.1313 - accuracy: 0.0091 - mae: 1.7550
Epoch 8/10
62/62 [==============================] - 0s 2ms/step - loss: 2.8105 - accuracy: 0.0091 - mae: 1.4165
Epoch 9/10
62/62 [==============================] - 0s 2ms/step - loss: 2.7867 - accuracy: 0.0091 - mae: 1.3951
Epoch 10/10
55/55 [==============================] - 0s 1ms/step - loss: 9.1296 - accuracy: 0.0086 - mae: 2.5197
Epoch 1/10
62/62 [==============================] - 0s 2ms/step - loss: 4.6109 - accuracy: 0.0091 - mae: 1.8325
Epoch 2/10
62/62 [==============================] - 0s 2ms/step - loss: 2.2226 - accuracy: 0.0091 - mae: 1.2355
Epoch 3/10
62/62 [==============================] - 0s 2ms/step - loss: 2.9657 - accuracy: 0.0091 - mae: 1.4120
Epoch 4/10
62/62 [==============================] - 0s 2ms/step - loss: 16.1235 - accuracy: 0.0084 - mae: 3.7274
Epoch 5/10
62/62 [==============================] - 0s 2ms/step - loss: 9.4079 - accuracy: 0.0091 - mae: 2.7691
E

In [23]:
#Ahora que ya hemos conseguido una perdida menor a la que queriamos, evaluamos para volver a comprobar que la hemos conseguido
red_bikes.evaluate(attr_prueba, obj_prueba)

55/55 [==============================] - 0s 1ms/step - loss: 0.8583 - accuracy: 0.0086 - mae: 0.7558


[0.8583170771598816, 0.008630610071122646, 0.7557636499404907]

In [24]:
#Ahora mismo comprobaremos manualmente si las predicciones de la red se parecen a lo que esperabamos utilizando los ultimos registros con los cuales no se ha entrenado

X=attr[17368]
predicciones = red_bikes.predict(X,verbose=0)
print('Esperado 203:',predicciones)
X=attr[17369]
predicciones = red_bikes.predict(X,verbose=0)
print('Esperado 247:',predicciones)
X=attr[17370]
predicciones = red_bikes.predict(X,verbose=0)
print('Esperado 315:',predicciones)
X=attr[17371]
predicciones = red_bikes.predict(X,verbose=0)
print('Esperado 214:',predicciones)
X=attr[17372]
predicciones = red_bikes.predict(X,verbose=0)
print('Esperado 164:',predicciones)
X=attr[17373]
predicciones = red_bikes.predict(X,verbose=0)
print('Esperado 122:',predicciones)
X=attr[17374]
predicciones = red_bikes.predict(X,verbose=0)
print('Esperado 119:',predicciones)
X=attr[17375]
predicciones = red_bikes.predict(X,verbose=0)
print('Esperado 89:',predicciones)
X=attr[17376]
predicciones = red_bikes.predict(X,verbose=0)
print('Esperado 90:',predicciones)
X=attr[17377]
predicciones = red_bikes.predict(X,verbose=0)
print('Esperado 61:',predicciones)
X=attr[17378]
predicciones = red_bikes.predict(X,verbose=0)
print('Esperado 49:',predicciones)

Esperado 203: [[201.83995]]
Esperado 247: [[245.15059]]
Esperado 315: [[314.97772]]
Esperado 214: [[215.32133]]
Esperado 164: [[163.08638]]
Esperado 122: [[120.54212]]
Esperado 119: [[117.03235]]
Esperado 89: [[88.81276]]
Esperado 90: [[89.51311]]
Esperado 61: [[60.400764]]
Esperado 49: [[48.975437]]


In [25]:
#Ahora trabajaremos con el otro modelo que será un random forest
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
forest_bikes = RandomForestRegressor()#Creamos el randomForest
forest_bikes.fit(attr_entrenamiento, obj_entrenamiento)#Lo entrenamos
evaluaciones = forest_bikes.predict(attr_prueba)#Evaluamos el modelo
mse = mean_squared_error(obj_prueba, evaluaciones)#Calculamos la perdida
print("Error cuadrático medio (MSE) del modelo:", mse)

Error cuadrático medio (MSE) del modelo: 3.136223820483319


In [26]:
#Ahora mismo comprobaremos manualmente si las predicciones del random forest se parecen a lo que esperabamos utilizando los ultimos registros con los cuales no se ha entrenado
attr2 = attr[:, np.newaxis]
X=attr2[17368]
predicciones = forest_bikes.predict(X)
print('Esperado 203:',predicciones)
X=attr2[17369]
predicciones = forest_bikes.predict(X)
print('Esperado 247:',predicciones)
X=attr2[17370]
predicciones = forest_bikes.predict(X)
print('Esperado 315:',predicciones)
X=attr2[17371]
predicciones = forest_bikes.predict(X)
print('Esperado 214:',predicciones)
X=attr2[17372]
predicciones = forest_bikes.predict(X)
print('Esperado 164:',predicciones)
X=attr2[17373]
predicciones = forest_bikes.predict(X)
print('Esperado 122:',predicciones)
X=attr2[17374]
predicciones = forest_bikes.predict(X)
print('Esperado 119:',predicciones)
X=attr2[17375]
predicciones = forest_bikes.predict(X)
print('Esperado 89:',predicciones)
X=attr2[17376]
predicciones = forest_bikes.predict(X)
print('Esperado 90:',predicciones)
X=attr2[17377]
predicciones = forest_bikes.predict(X)
print('Esperado 61:',predicciones)
X=attr2[17378]
predicciones = forest_bikes.predict(X)
print('Esperado 49:',predicciones)

Esperado 203: [202.69]
Esperado 247: [247.28]
Esperado 315: [313.93]
Esperado 214: [214.17]
Esperado 164: [164.]
Esperado 122: [121.97]
Esperado 119: [119.12]
Esperado 89: [88.9]
Esperado 90: [89.95]
Esperado 61: [61.05]
Esperado 49: [48.98]


### Método LIME (Local interpretable model-agnostic explanations)

A continuación, se puede observar el pseudocódigo que se nos ha proporcionado sobre el método LIME y, justo debajo, la función explain_model() que define el método LIME

In [27]:
#Metodo LIME
import random
from sklearn.linear_model import Ridge
from sklearn.neural_network import MLPClassifier, MLPRegressor



"""
Partiremos de este seudocódigo para implementar el metodo
N es el n´umero de permutaciones a realizar
f es el modelo a explicar
X′ ← {} muestras perturbadas
R ← {} representaciones
W ← {} las distancias entre la muestra x y sus perturbaciones
for 1
to
N do
Selecciona k atributos aleatoriamente
x′ ← una perturbaci´on de x donde se perturban los k atributos anteriores.
w ← la distancia entre x y x′
r ← la representaci´on de x′ respecto a x
X′ ← X′ ∪ x′
R ← R ∪ r
W ← R ∪ w
end for
Y ′ ← f(X′) las predicciones de las perturbaciones
G ← modelo ridge entrenado con R para predecir Y ′ y ponderando cada
muestra con W
return los par´ametros de G

"""

def explain_model(f, x, N,M):
    #f es el modelo a explicar
    #X es una lista de ejemplos a los que se aplicará LIME
    #N es el número de permutaciones a realizar
    #M contiene todos los ejemplos
    #Si queremos que el algoritmo cumpla la metrica identidad necesitamos fijar semillas para la aleatoriedad
    random.seed(11)
    Xi = [] #Aqui guardaremos las muestras perturbadas
    R = []  #Aqui guardaremos las representaciones
    W = []  #Aqui guradaremos las distancias
    for i in range(N):
        k = random.randint(1, len(x)) #Escojo un número aletario que representa el número de los atributos a seleccionar
        perturbed_x = x.copy() #copio la muestra original
        for j in range(k): 
            perturbed_attr = random.randint(0,len(x)-1)#Voy escogiendo los atributos que perturbare de forma aleatoria
            mx = abs(max([a[perturbed_attr] for a in M])) #Para el atributo aletorio seleccionado calculo el valor máximo
            mn = abs(min([a[perturbed_attr] for a in M])) #Para el atributo aletorio seleccionado calculo el valor mínimo
            perturbed_x[perturbed_attr] = random.uniform(mn, mx) # pertubo el atributo con un valor aleatorio ente mn y mx para que el valor esté acotado
        w = abs(sum([x[attr] - perturbed_x[attr] for attr in range(0,len(x)-1)])) #Calculo la distancia entre el original y el perturbado
        r = [0  if perturbed_x[attr] == x[attr] else 1 for attr in range(0,len(x)-1)] #Calculo la representación del perturbado respecto al original utilizando un operador ternario
        Xi.append(perturbed_x) #Acumulo en la lista los perturbados
        R.append(r)#Acumulamos en la lista de representaciones
        W.append(w)#Añadimos las distancias a la lista
    Y_perturbed = []#Aqui guardaremos las predicciones de las perturbaciones
    for i in range(len(Xi)):
        y = 1
            #En los siguiente if y else comprobamos el tipo de modelo que tenemos ya que dependiendo del tipo necesita recibir los atributos de una forma u otra
        if isinstance(f, keras.Sequential):#Comprueba si el modelo es una red neuronal
            xi = Xi[i]
            y = f.predict(xi,verbose=0)#Hacemos la prediccion con verbose=0 para no cargar la salida de lineas generadas por keras
        else: #Si no, es un randomForest
            xi = Xi[i]
            array = np.array(xi)
            xi = array.reshape(1, -1)
            y = f.predict(xi)#RandomForest no tiene verbose porque no genera nada
                
            
        Y_perturbed.append(y) #Aplico el modelo f a los ejemplos perturbados
            
    Y_perturbed = np.squeeze(Y_perturbed) #Por alguna razón aparece el error Found array with dim 3. Estimator expected <= 2" y lo arreglamos con esta linea que quita una dimensión al array
    G = Ridge()
    G.fit(R, Y_perturbed, sample_weight=W)
    return G.coef_#Devolvemos los parametros


Realizamos diferentes pruebas para comprobar que el método LIME funciona

In [28]:
#Hacemos varias pruebas para comprobar que funciona el metodo LIME
atributivos = atributos[5809]

explain_model(red_evaluation,atributivos,10,atributos)

array([-0.00322883, -0.00421551,  0.00186016, -0.07594658,  0.02182944,
       -0.02031252, -0.07267537, -0.05448938, -0.06145743, -0.03808059,
        0.02494923,  0.03223302, -0.00592798, -0.01283636, -0.07644896,
       -0.04371356, -0.03757821, -0.08928532, -0.10255304,  0.02117564,
       -0.05149973, -0.02474185, -0.0570523 ,  0.06520938, -0.07267537,
       -0.02987767, -0.04193458,  0.0119205 , -0.01653893,  0.00371758])

In [29]:
#Hacemos varias pruebas para comprobar que funciona el metodo LIME
atributivos = atributos[5809]

explain_model(red_evaluation,atributivos,10,atributos)

array([-0.00322883, -0.00421551,  0.00186016, -0.07594658,  0.02182944,
       -0.02031252, -0.07267537, -0.05448938, -0.06145743, -0.03808059,
        0.02494923,  0.03223302, -0.00592798, -0.01283636, -0.07644896,
       -0.04371356, -0.03757821, -0.08928532, -0.10255304,  0.02117564,
       -0.05149973, -0.02474185, -0.0570523 ,  0.06520938, -0.07267537,
       -0.02987767, -0.04193458,  0.0119205 , -0.01653893,  0.00371758])

In [30]:
atributivos = atributos[5809]

explain_model(forest_evaluation,atributivos,10,atributos)

array([-0.05283786,  0.02952605, -0.04877692, -0.01673585,  0.00926816,
        0.03693013, -0.0449714 , -0.03159047, -0.03670487, -0.04254281,
        0.00054464, -0.01163405, -0.05001559,  0.02219033, -0.02992612,
       -0.0283699 , -0.02935254, -0.00773579, -0.01408649,  0.01558992,
       -0.02938148, -0.05154288, -0.01936983,  0.02125141, -0.0449714 ,
        0.03088491, -0.02333626,  0.02529608,  0.02188485, -0.04813164])

In [31]:
attri = attr[17378]

explain_model(red_bikes,attri,10,attr)

array([ -23.21559503, -104.81841321,  199.61633045,   78.05094404,
         37.73909576, -171.71929281,  134.54972229, -103.10081051,
         43.57056969,    2.84717117,  -59.5110729 ,  -51.275878  ,
        153.14000203])

In [32]:
attri = attr[17378]

explain_model(forest_bikes,attri,10,attr)

array([ -26.06960708,  -96.35780144,  190.28332459,   62.96544304,
         51.28716309, -161.15508979,  148.2079223 , -111.75693617,
         20.10040857,  -19.66325832,  -51.78567413,  -64.67320952,
        126.96687171])

### Métricas

Una vez implementado el método LIME, pasamos con las métricas que se encargan de comprobar si se está aplicando correctamente

#### Identidad

In [33]:
#En la definición de cada metrica se importarán las cosas necesarias por si solo se quiere probar una metrica
import numpy as np


def identidad(x1,x2,exp1,exp2):
    #Definición: El principio de identidad establece que objetos idénticos deben recibir explicaciones idénticas.

    res = True #Si los objetos no son identicos no hay que evaluar
    distance = np.linalg.norm(x1 - x2)#Calcula las distancias entre los dos ejemplos
    if(distance ==0):   #Solo comprobamos si las explicaciones sean identicas si los objetos son identicos
        res = np.linalg.norm(exp1 - exp2) ==0#Comprobamos si ambas explicaciones son iguales
    return res

In [34]:
#Hacemos una pequeña prueba para ver que la identidad funciona cogiendo dos veces el mimso ejemplo y viendo que se generan las 
#mismas explicaciones
ejemplo2=attr[17375]
ejemplo1 = attr[17375]
print(ejemplo2)
print(ejemplo1)
exp1 = explain_model(forest_bikes,ejemplo1,2,attr)
exp2 = explain_model(forest_bikes,ejemplo2,2,attr)
print(identidad(ejemplo1,ejemplo2,exp1,exp2))

[0.         1.         1.         0.86956522 0.         0.16666667
 1.         0.33333333 0.24489796 0.2576     0.6        0.19301751
 0.02179837 0.09142212]
[0.         1.         1.         0.86956522 0.         0.16666667
 1.         0.33333333 0.24489796 0.2576     0.6        0.19301751
 0.02179837 0.09142212]
True


Comprobamos que la métrica identidad funciona

In [35]:
#Para comprobarla recorreremos 256 ejemplos en los cuatro modelos y para cada ejemplo en cada modelo generaremos 
#la explicacion dos veces y comprobaremos que es la misma explicación
atributivos = atributos[5553:5809,:] #seleccionamos varias muestras, si cambiamos el número de la izquierda de los dos puntos:, podemos hacer que se utilizen más o menos muestras
sonIguales = True #Inicializamos la varibale
for x in atributivos: #Recorremos las muestras
    exp1 = explain_model(forest_evaluation,x,2,atributos) #Generamos una explicación para una muestra
    exp2 = explain_model(forest_evaluation,x,2,atributos) #Volvemos a generar una explicación para la misma muestra
    sonIguales = sonIguales and identidad(x,x,exp1,exp2) #Comprobamos si cumplen la identidad
    if(not sonIguales): #Cuando no se cumpla rompemos el bucle
        break
#Mostrarmos el resualdo de evaluar la metrica identidad
if(sonIguales):
    print("El modelo RandomForest de evaluación cumple la métrica identidad")
else:
    print("El modelo RandomForest de evaluación no cumple la métrica identidad")

#Repetimos el proceso para los otros tres modelos, para los modelos de bike, además, utilizamos otros datos, como es normal
    
sonIguales=True
for x in atributivos:
    exp1 = explain_model(red_evaluation,x,2,atributos)
    exp2 = explain_model(red_evaluation,x,2,atributos)
    sonIguales = sonIguales and identidad(x,x,exp1,exp2)
    if(not sonIguales):
        break

if(sonIguales):
    print("El modelo Red neuronal de evaluación cumple la métrica identidad")
else:
    print("El modelo Red neuronal de evaluación no cumple la métrica identidad")

X=attr[17122:17378,:]
sonIguales=True
for x in X:
    exp1 = explain_model(red_bikes,x,2,attr)
    exp2 = explain_model(red_bikes,x,2,attr)
    sonIguales = sonIguales and identidad(x,x,exp1,exp2)
    if(not sonIguales):
        break
    
if(sonIguales):
    print("El modelo Red neuronal de bikes cumple la métrica identidad")
else:
    print("El modelo Red neuronal de bikes no cumple la métrica identidad")
    
sonIguales=True
for x in X:
    exp1 = explain_model(forest_bikes,x,2,attr)
    exp2 = explain_model(forest_bikes,x,2,attr)
    sonIguales = sonIguales and identidad(x,x,exp1,exp2)
    if(not sonIguales):
        break
    
if(sonIguales):
    print("El modelo RandomForest de bikes cumple la métrica identidad")
else:
    print("El modelo RandomForest de bikes no cumple la métrica identidad")

El modelo RandomForest de evaluación cumple la métrica identidad
El modelo Red neuronal de evaluación cumple la métrica identidad
El modelo Red neuronal de bikes cumple la métrica identidad
El modelo RandomForest de bikes cumple la métrica identidad


#### Separabilidad

In [36]:
import numpy as np


def separabilidad(x1,x2,exp1,exp2):
#Definición:#Separabilidad: Objetos no idénticos no pueden tener explicaciones idénticas. Para simplificar, cada característica 
#tiene un nivel mínimo de importancia, positivo o negativo, en las predicciones.

    res = True#Iicializamos la variable
    distance = np.linalg.norm(x1 - x2)#Calcula las distancias entre los dos ejemplos
    if(distance >0):   #Si la distancia es distinta de cero comparamos las explicaciones
        res = not(np.linalg.norm(exp1 - exp2) ==0)#Si las explicaciones son distintas se cumple la separabilidad
    
    return res

In [37]:
#Hacemos una pequeña prueba para comprobar que la separabilidad funciona correctamente, comparando dos muestras distintas
#y comprobando que sus explicaciones también son distintas. Aunque se parezcan bastante
ejemplo2=attr[17376]
ejemplo1 = attr[17375]
print(ejemplo2)
print(ejemplo1)
exp1 = explain_model(forest_bikes,ejemplo1,2,attr)
exp2 = explain_model(forest_bikes,ejemplo2,2,attr)
print(separabilidad(ejemplo1,ejemplo2,exp1,exp2))

[0.         1.         1.         0.91304348 0.         0.16666667
 1.         0.         0.24489796 0.2576     0.6        0.19301751
 0.01907357 0.09367946]
[0.         1.         1.         0.86956522 0.         0.16666667
 1.         0.33333333 0.24489796 0.2576     0.6        0.19301751
 0.02179837 0.09142212]
True


Comprobamos que la métrica separabilidad funciona

In [38]:
#Para comprobarla recorreremos 256 ejemplos en los cuatro modelos y los comparamos con otro ejemplo
ejemplo1=atributos[5552]#Escogemos una muestra para compararla con todas las muestras de un conjunto
atributivos = atributos[5553:5809,:] #Escogemos un conjunto de muestras 
cumpleSeparabilidad = True #Inicializamos una variable
for x in atributivos: #Recorremos las muestras
    exp1 = explain_model(forest_evaluation,x,2,atributos)#Generamos una explicación de la muestra que hemos escogido
    exp2 = explain_model(forest_evaluation,ejemplo1,2,atributos)#Y una explicación para cada muestra del conjunto
    cumpleSeparabilidad=cumpleSeparabilidad and separabilidad(x,ejemplo1,exp1,exp2) #Comprobamos si se cumple la separabilidad
    if(not cumpleSeparabilidad): #Si deja de cumplirse nos saltamos el bucle
        break
#Evaluamos si se cumple la separabilidad
if(cumpleSeparabilidad):
    print("El modelo RandomForest de evaluación cumple la métrica separabilidad")
else:
    print("El modelo RandomForest de evaluación no cumple la métrica separabilidad")

#Repetimos el proceso con los otros tres modelos     
    
cumpleSeparabilidad = True
for x in atributivos:
    exp1 = explain_model(red_evaluation,x,2,atributos)
    exp2 = explain_model(red_evaluation,ejemplo1,2,atributos)
    cumpleSeparabilidad=cumpleSeparabilidad and separabilidad(x,ejemplo1,exp1,exp2)
    if(not cumpleSeparabilidad):
        break

if(cumpleSeparabilidad):
    print("El modelo Red neuronal de evaluación cumple la métrica separabilidad")
else:
    print("El modelo Red neuronal de evaluación no cumple la métrica separabilidad")

ejemplo2 = attr[17121]
X=attr[17122:17378,:]
cumpleSeparabilidad=True
for x in X:
    exp1 = explain_model(red_bikes,x,2,attr)
    exp2 = explain_model(red_bikes,ejemplo2,2,attr)
    cumpleSeparabilidad=cumpleSeparabilidad and separabilidad(x,ejemplo2,exp1,exp2)
    if(not cumpleSeparabilidad):
        break
    
if(cumpleSeparabilidad):
    print("El modelo Red neuronal de bikes cumple la métrica separabilidad")
else:
    print("El modelo Red neuronal de bikes no cumple la métrica separabilidad")
    
cumpleSeparabilidad=True
for x in X:
    exp1 = explain_model(forest_bikes,x,2,attr)
    exp2 = explain_model(forest_bikes,ejemplo2,2,attr)
    cumpleSeparabilidad=cumpleSeparabilidad and separabilidad(x,ejemplo2,exp1,exp2)
    if(not cumpleSeparabilidad):
        break
    
if(cumpleSeparabilidad):
    print("El modelo RandomForest de bikes cumple la métrica separabilidad")
else:
    print("El modelo RandomForest de bikes no cumple la métrica separabilidad")

El modelo RandomForest de evaluación cumple la métrica separabilidad
El modelo Red neuronal de evaluación cumple la métrica separabilidad
El modelo Red neuronal de bikes cumple la métrica separabilidad
El modelo RandomForest de bikes cumple la métrica separabilidad


#### Estabilidad

In [39]:
import numpy as np
from scipy.stats import spearmanr

def estabilidad(x1,x2,exp1,exp2,umbEj,umbExp):
    #Objetos similares deben tener explicaciones similares
    #Para esto calcularemos la correlacion de Spearman para los ejemplos, si superan el limite que se pasará a la función
    #Calcularemos la correlación de Spearman para las explicaciones y comprobaremos si supera el umbral, en ese caso, cumpliran la estabilidad
    res = True #Inicializamos una variable
    correlacionEj, _ = spearmanr(x1, x2) #Calculamos la correlación de spearman de dos muestras
    if(abs(correlacionEj) >= umbEj): #Si es mayor que el umbral para las muestras, comprobaremos si sus explicaciones son parecidas
        correlacionExp, _ = spearmanr(exp1,exp2) #Calculamos la correlación de spearman de las explicaciones
        res = abs(correlacionExp)>=umbExp #Si es superior que el umbral de las explicaciones, se cumple la estabilidad
    return res

In [40]:
#Haremos una pequeña prueba para comprobar la que estabilidad funciona correctamente
#Escogemos dos muestras parecidas
ejemplo2=attr[17376]
ejemplo1 = attr[17375]
print(ejemplo2)
print(ejemplo1)
#Calulamos la correlacion de las muestras 
print(spearmanr(ejemplo1, ejemplo2))
#Generamos sus explicaciones
exp1 = explain_model(forest_bikes,ejemplo1,2,attr)
exp2 = explain_model(forest_bikes,ejemplo2,2,attr)
#Calculamos la correlación de las explicaciones
print(spearmanr(exp1, exp2))
#Tras esto, observamos que si se cumple la estabilidad
print(estabilidad(ejemplo1,ejemplo2,exp1,exp2,0.75,0.75))#Utilizamos la función para ver que funciona correctamente

[0.         1.         1.         0.91304348 0.         0.16666667
 1.         0.         0.24489796 0.2576     0.6        0.19301751
 0.01907357 0.09367946]
[0.         1.         1.         0.86956522 0.         0.16666667
 1.         0.33333333 0.24489796 0.2576     0.6        0.19301751
 0.02179837 0.09142212]
SpearmanrResult(correlation=0.8770812523243259, pvalue=3.801615077865054e-05)
SpearmanrResult(correlation=1.0, pvalue=0.0)
True


Comprobamos que la métrica estabilidad funciona

In [41]:
#Para comprobarla recorreremos 256 ejemplos en los cuatro modelos y los comparamos con otro ejemplo, del cual sabemos que hay ejemplos muy similares

ejemplo1=atributos[5819]
atributivos = atributos[5553:5819,:]
cumpleEstabilidad = True
for x in atributivos:
    exp1 = explain_model(forest_evaluation,x,2,atributos)
    exp2 = explain_model(forest_evaluation,ejemplo1,2,atributos)
    cumpleEstabilidad=cumpleEstabilidad and estabilidad(x,ejemplo1,exp1,exp2,0.75,0.75)
    if(not cumpleEstabilidad):
        break
if(cumpleEstabilidad):
    print("El modelo RandomForest de evaluación cumple la métrica separabilidad")
else:
    print("El modelo RandomForest de evaluación no cumple la métrica separabilidad")

cumpleEstabilidad = True
for x in atributivos:
    exp1 = explain_model(red_evaluation,x,2,atributos)
    exp2 = explain_model(red_evaluation,ejemplo1,2,atributos)
    cumpleEstabilidad=cumpleEstabilidad and estabilidad(x,ejemplo1,exp1,exp2,0.75,0.75)
    if(not cumpleEstabilidad):
        break

if(cumpleEstabilidad):
    print("El modelo Red neuronal de evaluación cumple la métrica separabilidad")
else:
    print("El modelo Red neuronal de evaluación no cumple la métrica separabilidad")

ejemplo2 = attr[17376]
X=attr[17122:17378,:]
cumpleEstabilidad=True
for x in X:
    exp1 = explain_model(red_bikes,x,2,attr)
    exp2 = explain_model(red_bikes,ejemplo2,2,attr)
    cumpleEstabilidad=cumpleEstabilidad and estabilidad(x,ejemplo2,exp1,exp2,0.75,0.75)
    if(not cumpleEstabilidad):
        break
    
if(cumpleEstabilidad):
    print("El modelo Red neuronal de bikes cumple la métrica estabilidad")
else:
    print("El modelo Red neuronal de bikes no cumple la métrica estabilidad")
    
cumpleEstabilidad=True
for x in X:
    exp1 = explain_model(forest_bikes,x,2,attr)
    exp2 = explain_model(forest_bikes,ejemplo2,2,attr)
    cumpleEstabilidad=cumpleEstabilidad and estabilidad(x,ejemplo2,exp1,exp2,0.75,0.75)
    if(not cumpleEstabilidad):
        break
    
if(cumpleEstabilidad):
    print("El modelo RandomForest de bikes cumple la métrica estabilidad")
else:
    print("El modelo RandomForest de bikes no cumple la métrica estabilidad")

El modelo RandomForest de evaluación cumple la métrica separabilidad
El modelo Red neuronal de evaluación cumple la métrica separabilidad
El modelo Red neuronal de bikes cumple la métrica estabilidad
El modelo RandomForest de bikes cumple la métrica estabilidad


#### Selectividad

In [42]:
import numpy as np
from scipy import integrate
from scipy.integrate import trapz

def selectividad(x, model,exp):
    #Calcularemos el area bajo la curva a partir de ir quitando las características más importantes
    y = x.copy() #Generamos una copia de la muestra recibida
    errores = [] #Aqui acumularemos los errores
    prediccionOriginal = 0 
    iteraciones= [] #Aqui guardaremos el número de iteraciones, necesario para calcular la integral
    if isinstance(model, keras.Sequential): #Dependiendo del modelo que reciba, necesita un tipo de datos u otro 
            prediccionOriginal = model.predict(x,verbose = 0) #Predecimos si es una red neuronal
    else:
            array = np.array(x)
            x = array.reshape(1, -1)
            prediccionOriginal = model.predict(x) #Si es random forest, utilizamos los datos de forma distinta
    
    for i in range(len(exp)):#Recorremos el número de parametros de las explicaciones
        masImportante = max(exp) #Busacmos la mayor importante
        atributo = np.where(exp == masImportante)[0] # Averiguamos en que posición se encuentra esta importancia
        y[atributo] = 0 #modificamos y la característica más importante en este momento
        exp[atributo] = -999999999999999999 #Cambiamos la importancia de esta característica haciendo que tenga un valor muy negativo para que nunca sea la más importante
         #Hacemos una predicción con la muestra modificada:
        if isinstance(model, keras.Sequential):
            z = model.predict(y,verbose=0)
            
        else:
            array = np.array(y)
            y2 = array.reshape(1, -1)
            z = model.predict(y2)
        iteraciones.append(i)
        errores.append(abs(prediccionOriginal-z))#Acumulamos el error
    
    #Cambiamos las dimensiones de ambos np.array, esto ocurre al meter las predicciones en una array, se modifica la dimensión de este
    errores = np.squeeze(errores) 
    res = trapz(iteraciones,errores) #Calculamos el area bajo la cruva
    return res


Comprobamos que la métrica selectividad funciona

In [43]:
#Para probar la selectividad sobre el metodo lime, recorremos varias muestras y veremos cuales son sus selectividades y calcularemos una media
atributivos = atributos[5810:5819,:]#seleccionamos varias muestras, si cambiamos el número de la izquierda de los dos puntos:, podemos hacer que se utilizen más o menos muestras

selectividades = [] #Aqui acumularemos las selectividades

for i in range(len(atributivos)): #Recorremos las muestras
    exp = explain_model(red_evaluation,atributivos[i],2,atributos) #Generamos una explicación para cada muestra
    sel = selectividad(atributivos[i],red_evaluation,exp) #Calulamos la selectividad de esta muestra
    selectividades.append(sel)#Acumulamos la selectividades
    
print("Estas son las selectividades del modelo Red Evaluación ",selectividades) #Mostramos las selectividades
print("La selectividad media del modelo Red Evaluación es ", np.mean(selectividades)) #Mostramos la media

#Repetimos el proceso para los demás modelos

selectividades = []

for i in range(len(atributivos)):
    exp = explain_model(forest_evaluation,atributivos[i],2,atributos)
    sel = selectividad(atributivos[i],forest_evaluation,exp)
    selectividades.append(sel)
    
print("Estas son las selectividades del modelo RandomForest Evaluación ",selectividades)
print("La selectividad media del modelo RandomForest Evaluación es ", np.mean(selectividades))

X = attr[17360:17378,:]   

selectividades = []
for i in range(len(X)):
    exp = explain_model(red_bikes,X[i],2,attr)
    sel = selectividad(X[i],red_bikes,exp)
    selectividades.append(sel)
    
print("Estas son las selectividades del modelo Red bikes ",selectividades)
print("La selectividad media del modelo Red bikes es ", np.mean(selectividades))

selectividades = []

for i in range(len(X)):
    exp = explain_model(forest_bikes,X[i],2,attr)
    sel = selectividad(X[i],forest_bikes,exp)
    selectividades.append(sel)
    
print("Estas son las selectividades del modelo RandomForest bikes ",selectividades)
print("La selectividad media del modelo RandomForest bikes es ", np.mean(selectividades))


Estas son las selectividades del modelo Red Evaluación  [0.5318917632102966, 0.6038385629653931, 0.03473943471908569, 0.41568177938461304, 0.09767711162567139, 0.03473943471908569, 3.5490570068359375, 3.697249412536621, 0.05347168445587158]
La selectividad media del modelo Red Evaluación es  1.0020384656058416
Estas son las selectividades del modelo RandomForest Evaluación  [0.68, 0.64, 0.0, 0.6350000000000001, 0.0, 0.0, 3.935, 3.9749999999999996, 0.0]
La selectividad media del modelo RandomForest Evaluación es  1.096111111111111
Estas son las selectividades del modelo Red bikes  [0.2838735580444336, 1.1428718566894531, 0.554107666015625, 4.2552490234375, 5.9341583251953125, 15.272884368896484, 21.146621704101562, 24.24462127685547, 18.194976806640625, 29.652061462402344, 36.16858673095703, 15.243057250976562, 5.657196044921875, 4.164730072021484, 4.211971282958984, 3.7576217651367188, 2.7836685180664062, 6.261701583862305]
La selectividad media del modelo Red bikes es  11.051664405398

#### Coherencia

In [44]:
def coherencia(mae1, mae2):
    #Se calcula la diferencia entre el error de predicción (mae1) sobre la señal original y el error de predicción mae2
    #de una nueva señal donde se eliminan las características no importantes.
    alpha_i = abs(mae1 - mae2)
    return alpha_i


Comprobamos que la métrica coherencia funciona

In [45]:
#Para probar la coherencia lo que haremos será para varias muestras, generar una predicción, generar su explicación 
#y a partir de su explicación quitaremos los N atributos menos importantes y generaremos otra explicación, a partir de ambas predicciones 
#calcularemos dos MAE y con ello calcularemos la coherencia
from sklearn.metrics import mean_absolute_error

#Escogemos un conjunto de musetras junto a sus valores objetivo
atributivos = atributos[5810:5819,:]
objetivos = objetivo[5810:5819]

N=2#Definimos el número de atributos que quitaremos
coherencias = []


for i in range(len(atributivos)):#Recorremos las muestras
    x = atributivos[i] 
    y = x.copy() #Generamos una copia de la muestra
    exp1 = explain_model(red_evaluation,x,2,atributos) #Generamos una explicación para la muestra
    prediccion1 = red_evaluation.predict(x,verbose=0) #Hacemos una predicción para la muestra
    errp = abs(objetivos[i]-prediccion1) #Calculamos el error entre la predicción y el objetivo
   
    for j in range(N): #Recorremos el número de atributos que queremos quitar
        menosImportante = min(exp1) #Cogemos la menor importancia
        atributo = np.where(exp1 == menosImportante)[0] #Buscamos a que atributo hace referencia
        y[atributo] = 0 #Quitamos el atributo
        exp1[atributo] = 99999999999999999 #Cambiamos el valor de esta imporatncia para que nunca más sea  la menor
    prediccion2 = red_evaluation.predict(y,verbose=0) #Generemos una predicción con la muestra modificada
   
    errexp = abs(objetivos[i]-prediccion2) #Calculamos el error de la muestra modificada
    res = coherencia(errp,errexp)
    coherencias.append(res) #Calculamos la coherencia y la metemos en su lista

#Cambiamos las dimensiones del array y los mostramos
coherencias = np.squeeze(coherencias)
print("El modelo Red Neuronal de evaluación tiene unas coherencias de ",coherencias)
print("El modelo Red Neuronal de evaluación tiene una coherencia media de de ",np.mean(coherencias))


#Repetimos el proceso con el resto de modelos

N=2
coherencias = []

for i in range(len(atributivos)):
    x = atributivos[i]
    y = x.copy()
    exp1 = explain_model(forest_evaluation,x,2,atributos)
    x = x.reshape(1, -1)
    prediccion1 = forest_evaluation.predict(x)
    errp = abs(objetivos[i]-prediccion1)
    
   
    for i in range(N):
        menosImportante = min(exp1)
        atributo = np.where(exp1 == menosImportante)[0]
        y[atributo] = 0
        exp1[atributo] = 999999999999999999
    y = y.reshape(1, -1)
    prediccion2 = forest_evaluation.predict(y)
      
    errexp = abs(objetivos[i]-prediccion2)
    res = coherencia(errp,errexp)
    coherencias.append(res) 
    
coherencias = np.squeeze(coherencias)
print("El modelo RandomForest de evaluación tiene unas coherencias de ",coherencias)
print("El modelo RandomForest de evaluación tiene una coherencia media de de ",np.mean(coherencias))


X=attr[17360:17378,:]
objs = obj[17360:17378]


N=2
coherencias = []


for i in range(len(X)):
    x = X[i]
    y = x.copy()
    exp1 = explain_model(red_bikes,x,2,attr)
    prediccion1 = red_bikes.predict(x,verbose=0)
    errp = abs(objs[i]-prediccion1)
    
   
    for i in range(N):
        menosImportante = min(exp1)
        atributo = np.where(exp1 == menosImportante)[0]
        y[atributo] = 0
        exp1[atributo] = 99999999999999999
    prediccion2 = red_bikes.predict(y,verbose=0)
       
    errexp = abs(objs[i]-prediccion2)
    res = coherencia(errp,errexp)
    coherencias.append(res) 


coherencias = np.squeeze(coherencias)
print("El modelo Red Neuronal de bikes tiene unas coherencias de ",coherencias)
print("El modelo Red Neuronal de bikes tiene una coherencia media de de ",np.mean(coherencias))    


N=2
coherencias = []

for i in range(0,len(X)):
    x = X[i]
    y = x.copy()
    exp1 = explain_model(forest_bikes,x,2,attr)
    x = x.reshape(1, -1)
    prediccion1 = forest_bikes.predict(x)
    errp = abs(objs[i]-prediccion1)
    
   
    for i in range(N):
        menosImportante = min(exp1)
        atributo = np.where(exp1 == menosImportante)[0]
        y[atributo] = 0
        exp1[atributo] = 999999999999999999
    y = y.reshape(1, -1)
    prediccion2 = forest_bikes.predict(y)
    errexp = abs(objs[i]-prediccion2)
    res = coherencia(errp,errexp)
    coherencias.append(res) 
    

    
coherencias = np.squeeze(coherencias)
print("El modelo RandomForest de bikes tiene unas coherencias de ",coherencias)
print("El modelo RandomForest de bikes tiene una coherencia media de de ",np.mean(coherencias))


El modelo Red Neuronal de evaluación tiene unas coherencias de  [0.79724026 0.6566241  0.06947887 0.8140454  0.24250627 0.06947887
 3.322833   3.3992515  0.10694337]
El modelo Red Neuronal de evaluación tiene una coherencia media de de  1.0531557
El modelo RandomForest de evaluación tiene unas coherencias de  [0.47 0.47 2.   0.47 1.8  2.   1.53 1.58 2.  ]
El modelo RandomForest de evaluación tiene una coherencia media de de  1.3688888888888888
El modelo Red Neuronal de bikes tiene unas coherencias de  [3.04710827e+01 3.87001038e-02 4.37824860e+01 1.45621979e+02
 1.01936447e+02 4.66019058e+01 7.21863861e+01 1.30352829e+02
 1.21622208e+02 1.41334778e+02 2.05929123e+02 1.40640076e+02
 1.09050064e+02 7.02299118e+01 6.62109451e+01 4.14626770e+01
 4.36127777e+01 7.85855103e+00]
El modelo Red Neuronal de bikes tiene una coherencia media de de  84.38571
El modelo RandomForest de bikes tiene unas coherencias de  [3.1000e+01 5.0000e-02 4.2460e+01 1.5024e+02 1.0754e+02 4.6950e+01
 7.4950e+01 1.36

#### Completitud

In [46]:
def completitud(ei, pi):
    #Evalúa el porcentaje de error de explicación con respecto al error de predicción.
    gamma_i = ei / pi
    return gamma_i

Comprobamos que la métrica completitud funciona

In [47]:
#Para evaluar la completitud, recorreremos varias muestras y calcularemos el error de prediccion y el de explicacion 
#para calcular la completitud
from sklearn.metrics import mean_absolute_error

#Escogemos varias muestras y sus objetivos
atributivos = atributos[5810:5819,:]
objetivos = objetivo[5810:5819]
#Necesitaremos además, los atributos sin normalizar
atributos2 = evaluation.loc[:, 'class':'Q27']
atributos2 = atributos2.to_numpy()
atributos2 = atributos2[5810:5819,:]
#Necesitaremos dos arrays para guardar las predicciones
predicciones1 = []
predicciones2 = []

for j in range(len(atributivos)):#Reccoremos las muestras
    x = atributivos[j]
    pred1 = red_evaluation.predict(x,verbose = 0) #Generamos una prediccion por muestra
    predicciones1.append(pred1)
    y = 0
    exp1 = explain_model(red_evaluation,x,2,atributos) #Generamos una prediccion por muestra
    atributo2 = atributos2[j]
    for i in range(len(exp1)): #Calculamos la prediccion de la esplicación, esto lo hacemos multiplicando para 
        y += exp1[i]*atributo2[i]#cada atributo, el atributo sin normalizar por la importancia de este atributo
    predicciones2.append(y)

predicciones1 = np.squeeze(predicciones1)  
predicciones2 = np.squeeze(predicciones2) 
mae1 = mean_absolute_error(objetivos, predicciones1)#Calculamos el error de predicción
mae2 = mean_absolute_error(objetivos, predicciones2)#Calculamos el error de explicaciones

res =  completitud(mae2,mae1)#Calculamos la completitud
#Mostramos la completitud
print("El modelo Red Neuronal de evaluación tiene unas completitudes de ",res)

#Repetimos el proceso para el resto de modelos

predicciones1 = []
predicciones2 = []
for j in range(len(atributivos)):
    x = atributivos[j]
    exp1 = explain_model(forest_evaluation,x,2,atributos)
    x = x.reshape(1,-1)
    pred1 = forest_evaluation.predict(x)
    predicciones1.append(pred1)
    y = 0
    atributo2 = atributos2[j]
    for i in range(len(exp1)):
        y += exp1[i]*atributo2[i]
    predicciones2.append(y)

predicciones1 = np.squeeze(predicciones1)  
predicciones2 = np.squeeze(predicciones2) 
mae1 = mean_absolute_error(objetivos, predicciones1)
mae2 = mean_absolute_error(objetivos, predicciones2)

res =  completitud(mae2,mae1)
print("El modelo RandomForest de evaluación tiene una completitud de ",res)


X = attr[17360:17378,:]
objs = obj[17360:17378]

attr2 = bikes.loc[:, 'season':'registered']
attr2 = attr2.to_numpy()
attr2 = attr2[17360:17378,:]
predicciones1 = []
predicciones2 = []
for j in range(len(X)):
    x = X[j]
    pred1 = red_bikes.predict(x,verbose = 0)
    predicciones1.append(pred1)
    y = 0
    exp1 = explain_model(red_bikes,x,2,attr)
    atributo2 = attr2[j]
    for i in range(len(exp1)):
        y += exp1[i]*atributo2[i]
    predicciones2.append(y)

predicciones1 = np.squeeze(predicciones1)  
predicciones2 = np.squeeze(predicciones2) 
mae1 = mean_absolute_error(objs, predicciones1)
mae2 = mean_absolute_error(objs, predicciones2)

res =  completitud(mae2,mae1)
print("El modelo Red Neuronal de bikes tiene una completitud de ",res)

predicciones1 = []
predicciones2 = []
for j in range(len(X)):
    x = X[j]
    exp1 = explain_model(forest_bikes,x,2,atributos)
    x = x.reshape(1,-1)
    pred1 = forest_bikes.predict(x)
    predicciones1.append(pred1)
    y = 0
    atributo2 = attr2[j]
    for i in range(len(exp1)):
        y += exp1[i]*atributo2[i]
    predicciones2.append(y)

predicciones1 = np.squeeze(predicciones1)  
predicciones2 = np.squeeze(predicciones2) 
mae1 = mean_absolute_error(objs, predicciones1)
mae2 = mean_absolute_error(objs, predicciones2)

res =  completitud(mae2,mae1)
print("El modelo RandomForest de bikes tiene una completitud de ",res)
    

El modelo Red Neuronal de evaluación tiene unas completitudes de  15.00829161723522
El modelo RandomForest de evaluación tiene una completitud de  372.5182743787257
El modelo Red Neuronal de bikes tiene una completitud de  360.9238343313193
El modelo RandomForest de bikes tiene una completitud de  793.0465426152326


#### Congruencia

In [48]:
import numpy as np

def congruencia(coherencias):
    #Esta métrica ayuda a capturar la variabilidad de la coherencia.
    coherencia_media = np.mean(coherencias) #Calculamos la coherencia media
    res = (sum((c - coherencia_media)**2 for c in coherencias)/len(coherencias))**0.5 #A partir de ella, calculamos la variabilidad
    return res

Comprobamos que la métrica congruencia funciona

In [49]:
#Para comprobar la congruencia, calcularemos la coherencia de varias muestras y apartir de eso se calculará la congruencia
from sklearn.metrics import mean_absolute_error
import numpy as np

#Cogemos varias meustras y sus valores objetivos
atributivos = atributos[5810:5819,:]
objetivos = objetivo[5810:5819]

#Defnimos el número de características a eliminar
N=2
coherencias = []

for i in range(len(atributivos)): #Recorremos las muestras
    x = atributivos[i]
    y = x.copy()
    exp1 = explain_model(red_evaluation,x,2,atributos) #Generamos una explicacion para cada muestra
    prediccion1 = red_evaluation.predict(x,verbose=0) #Hacemos una predicción
    errp = abs(objetivos[i]-prediccion1) #Calculamos el error de predicción
   
    for j in range(N): #Eliminaremos N características
        menosImportante = min(exp1) #Cogemos la menor importancia
        atributo = np.where(exp1 == menosImportante)[0] #Vemos a que atributo pertenece
        y[atributo] = 0 #Ponemos ese atributo a cero
        exp1[atributo] = 99999999999999999 #Cambiamos la importancia de esta característica para que nunca más sea la mínima
    prediccion2 = red_evaluation.predict(y,verbose=0) #Hacemos una predicción con la muestra modificada
   
    errexp = abs(objetivos[i]-prediccion2) #Calculamos el error de la predicción de la muestra modificada
    coherencias.append(coherencia(errp,errexp))
    

coherencias = list(coherencias)
congr = congruencia(coherencias) #Calculamos la congruencia

    
#Mostramos la congruencia
print("El modelo Red Neuronal de evaluación tiene una congruencia de ",congr)

#Repetimos el proceso para los demás modelos

N=2
coherencias = []

for i in range(len(atributivos)):
    x = atributivos[i]
    y = x.copy()
    exp1 = explain_model(forest_evaluation,x,2,atributos)
    x = x.reshape(1, -1)
    prediccion1 = forest_evaluation.predict(x)
    errp = abs(objetivos[i]-prediccion1)
    
   
    for i in range(N):
        menosImportante = min(exp1)
        atributo = np.where(exp1 == menosImportante)[0]
        y[atributo] = 0
        exp1[atributo] = 999999999999999999
    y = y.reshape(1, -1)
    prediccion2 = forest_evaluation.predict(y)
      
    errexp = abs(objetivos[i]-prediccion2)
    coherencias.append(coherencia(errp,errexp))
    
coherencias = list(coherencias)
congr = congruencia(coherencias)

print("El modelo RandomForest de evaluación tiene una congruencia de ",congr)

X=attr[17360:17378,:]
objs = obj[17360:17378]


N=2
coherencias = []


for i in range(len(X)):
    x = X[i]
    y = x.copy()
    exp1 = explain_model(red_bikes,x,2,attr)
    prediccion1 = red_bikes.predict(x,verbose=0)
    errp = abs(objs[i]-prediccion1)
    
   
    for i in range(N):
        menosImportante = min(exp1)
        atributo = np.where(exp1 == menosImportante)[0]
        y[atributo] = 0
        exp1[atributo] = 99999999999999999
    prediccion2 = red_bikes.predict(y,verbose=0)
       
    errexp = abs(objs[i]-prediccion2)
    coherencias.append(coherencia(errp,errexp))

coherencias = list(coherencias)
congr = congruencia(coherencias)    

print("El modelo Red Neuronal de bikes tiene una congruencia de ",congr)


N=2
coherencias = []

for i in range(0,len(X)):
    x = X[i]
    y = x.copy()
    exp1 = explain_model(forest_bikes,x,2,attr)
    x = x.reshape(1, -1)
    prediccion1 = forest_bikes.predict(x)
    errp = abs(objs[i]-prediccion1)
    
   
    for i in range(N):
        menosImportante = min(exp1)
        atributo = np.where(exp1 == menosImportante)[0]
        y[atributo] = 0
        exp1[atributo] = 999999999999999999
    y = y.reshape(1, -1)
    prediccion2 = forest_bikes.predict(y)
    errexp = abs(objs[i]-prediccion2)
    coherencias.append(coherencia(errp,errexp))
    
coherencias = list(coherencias)
congr = congruencia(coherencias)

print("El modelo RandomForest de bikes tiene una congruencia de ",congr)


El modelo Red Neuronal de evaluación tiene una congruencia de  [[1.2659303]]
El modelo RandomForest de evaluación tiene una congruencia de  [0.65619178]
El modelo Red Neuronal de bikes tiene una congruencia de  [[53.998215]]
El modelo RandomForest de bikes tiene una congruencia de  [56.01493183]
